In [120]:
import numpy as np
import pandas as pd
from shapely.geometry import Point
from rtree import index
from GISio import shp2df, df2shp, get_values_at_points

In [121]:
newlines = '../Examples/data/added_lines.shp'
sfrlines = '../Examples/data/SFRlines.shp'
dem = 'D:/ATLData/BR/BadRiver/grid/dem/dem_utm_ft/w001001.adf'

In [122]:
nl = shp2df(newlines)
sfr = shp2df(sfrlines)
tol = 100
nsegstart = sfr.segment.astype(int).max() + 1


reading ../Examples/data/added_lines.shp...
--> building dataframe... (may take a while for large shapefiles)

reading ../Examples/data/SFRlines.shp...
--> building dataframe... (may take a while for large shapefiles)


In [123]:
start_cds = [(g.xy[0][0], g.xy[1][0]) for g in nl.geometry]
end_cds = [(g.xy[0][-1], g.xy[1][-1]) for g in nl.geometry]
sfr_start_cds = [(g.xy[0][0], g.xy[1][0]) for g in sfr.geometry]

In [124]:
max_elevs = get_values_at_points(dem, start_cds)
min_elevs = get_values_at_points(dem, end_cds)

In [125]:
nl = pd.DataFrame(nl.geometry)
nl['segment'] = np.arange(nsegstart, nsegstart+len(nl))
nl['elevMax'] = max_elevs
nl['elevMin'] = min_elevs
nl['upsegs'] = [[]] * len(nl)
nl['outseg'] = 0

In [137]:
# get index of nearest start to each end
def get_nearest(starts, ends):
    idx = index.Index()
    for i, start in enumerate(starts):
        idx.insert(i, start)
    return [list(idx.nearest(end, 2))[0] if list(idx.nearest(end, 1))[0] != i
            else list(idx.nearest(end, 2))[1]
            for i, end in enumerate(ends)]


print 'routing new lines...'
nearest_start = get_nearest(start_cds, end_cds)

# record the preliminary seg. number of nearest start if within tol
nl['outseg'] = [nl.segment[n] if Point(*start_cds[n]).distance(Point(*ends_cds[i])) < tol 
                 else 0 for i, n in enumerate(nearest_start)]

nl['upsegs'] = [nl.segment[nl.outseg == s].tolist() for s in nl.segment]

routing new lines...


In [138]:
print 'routing new lines to SFR...'

is_outlet = nl.outseg.values == 0
new_lines_outlet_cds = map(tuple, np.array(end_cds)[is_outlet])

# get index of nearest start to each end
nearest_sfr = get_nearest(sfr_start_cds, new_lines_outlet_cds)

# record the preliminary seg. number of nearest start if within tol
nl.loc[is_outlet, 'outseg'] = [sfr.segment[n] 
                               if Point(*sfr_start_cds[n]).distance(\
                                  Point(*new_lines_outlet_cds[i])) < tol 
                               else 0 for i, n in enumerate(nearest_sfr)]

routing new lines to SFR...


In [139]:
df2shp(nl, 'temp/junk.shp', epsg=26715)

writing temp/junk.shp...


In [145]:
[Point(*sfr_start_cds[n]).distance(\
Point(*new_lines_outlet_cds[i])) 
for i, n in enumerate(nearest_sfr)]

[30.931128978382517,
 66.42850017582501,
 44.026341914120756,
 35.69563040706045,
 85.70023138569788,
 76.34531215790344,
 54.824131959398755,
 25.75964296381895,
 59.51208598342562,
 10.832650014948323,
 99.58368702124653,
 29.576890262867256,
 65.41116556855103,
 271.7038636135656,
 58.22844255944167,
 75.74442817947747,
 43.53502837533221,
 97.9263938885384,
 42.04153101217789,
 57.90288211351542,
 1029.497285560213,
 27.174309998463546,
 119.88181008971353,
 77.02044993997885,
 68.9077294579952,
 50.2912156861218,
 38.286644892390825,
 41.34612415155738,
 28.964544361385563,
 77.91637608006417,
 74.63495311141715,
 63.10083406893315,
 62.10678632674487,
 62.75179152755675,
 29.305517998591004,
 271.26877316733606,
 102.02944325452265,
 63.756383209940765,
 17.293891263515214,
 57.19268189055897,
 40.29534580473399,
 98.04355766962823,
 12.799579661857168,
 1894.8629675167558,
 5.600683603881446,
 164.8448293417493,
 85.80996567641861,
 91.54179567730884,
 55.52508913701957,
 716.54

In [135]:
idx = index.Index()
for i, start in enumerate(start_cds):
    idx.insert(i, start)
[list(idx.nearest(end, 2))[0] if list(idx.nearest(end, 1))[0] != i
        else list(idx.nearest(end, 2))[1]
        for i, end in enumerate(end_cds)]

[127L,
 68L,
 105L,
 5L,
 70L,
 103L,
 99L,
 83L,
 24L,
 23L,
 57L,
 49L,
 77L,
 69L,
 76L,
 79L,
 95L,
 12L,
 97L,
 116L,
 110L,
 71L,
 113L,
 2L,
 8L,
 85L,
 77L,
 85L,
 114L,
 33L,
 39L,
 24L,
 63L,
 92L,
 71L,
 52L,
 10L,
 68L,
 14L,
 30L,
 133L,
 103L,
 5L,
 9L,
 110L,
 64L,
 60L,
 129L,
 117L,
 136L,
 1L,
 17L,
 35L,
 26L,
 93L,
 89L,
 78L,
 31L,
 120L,
 50L,
 112L,
 67L,
 59L,
 25L,
 45L,
 73L,
 49L,
 80L,
 58L,
 98L,
 55L,
 21L,
 12L,
 92L,
 15L,
 3L,
 99L,
 12L,
 38L,
 15L,
 2L,
 134L,
 6L,
 7L,
 112L,
 27L,
 50L,
 110L,
 122L,
 55L,
 40L,
 115L,
 118L,
 29L,
 0L,
 16L,
 126L,
 135L,
 61L,
 6L,
 89L,
 35L,
 58L,
 41L,
 106L,
 42L,
 108L,
 21L,
 22L,
 52L,
 44L,
 82L,
 60L,
 126L,
 28L,
 28L,
 19L,
 48L,
 15L,
 43L,
 58L,
 64L,
 88L,
 37L,
 83L,
 90L,
 132L,
 0L,
 90L,
 90L,
 107L,
 133L,
 1L,
 40L,
 88L,
 97L,
 49L]

In [132]:
[i for i, end in enumerate(end_cds)]

[0,
 1,
 2,
 3,
 4,
 5,
 6,
 7,
 8,
 9,
 10,
 11,
 12,
 13,
 14,
 15,
 16,
 17,
 18,
 19,
 20,
 21,
 22,
 23,
 24,
 25,
 26,
 27,
 28,
 29,
 30,
 31,
 32,
 33,
 34,
 35,
 36,
 37,
 38,
 39,
 40,
 41,
 42,
 43,
 44,
 45,
 46,
 47,
 48,
 49,
 50,
 51,
 52,
 53,
 54,
 55,
 56,
 57,
 58,
 59,
 60,
 61,
 62,
 63,
 64,
 65,
 66,
 67,
 68,
 69,
 70,
 71,
 72,
 73,
 74,
 75,
 76,
 77,
 78,
 79,
 80,
 81,
 82,
 83,
 84,
 85,
 86,
 87,
 88,
 89,
 90,
 91,
 92,
 93,
 94,
 95,
 96,
 97,
 98,
 99,
 100,
 101,
 102,
 103,
 104,
 105,
 106,
 107,
 108,
 109,
 110,
 111,
 112,
 113,
 114,
 115,
 116,
 117,
 118,
 119,
 120,
 121,
 122,
 123,
 124,
 125,
 126,
 127,
 128,
 129,
 130,
 131,
 132,
 133,
 134,
 135,
 136]